In [2]:
from sklearn.model_selection import KFold
import pickle
import pandas as pd
import os.path
import numpy as np
# path dictionary
path_data_folder = "5.9.23/"
path_dictionary = {
    '20.7': path_data_folder + "twentypointseven",
    '21.0': path_data_folder + "twentyonepointzero_1",
    '21.2': path_data_folder + "twentyonepointtwo_1",
    '21.5': path_data_folder + "twentyonepointfive_1",
    '21.7': path_data_folder + "TwentyonepointseevendegreeC_1",
    '21.8': path_data_folder + "twentypointeight_1"
}
# separator in this file is tab
# label for entire file is the temperature
# frames = pd.read_csv("5.9.23/twentypointseven", sep="\t", header=None)
# # <- pandas index is [column][row]

In [3]:
# the data are (1000 * 1) column vectors.
# in the file, there are 1000 lines, each with n numbers, 
# where n = number of data vectors
def load_data(filename_dictionary):
    X_data = [] # data
    y_data = [] # label
    for filename, filepath in filename_dictionary.items():
        print(f"reading file:        {filepath}")
        X_in_this_file = pd.read_csv(filepath, sep="\t", header=None)
        value = float(filename)
        print(f"\ttemperature value: {value}")
        number_of_examples = X_in_this_file.shape[1]
        y_in_this_file = np.zeros(shape=(1, number_of_examples)) + value
        y_in_this_file = pd.DataFrame(y_in_this_file)
        # default column setting is NO array, 
        # need to make it array to use list of indices!
        y_in_this_file.columns = np.asarray(range(y_in_this_file.shape[1]))
        X_data.append(X_in_this_file)
        y_data.append(y_in_this_file)
    X_data = pd.concat(X_data, axis=1, ignore_index=True)
    y_data = pd.concat(y_data, axis=1, ignore_index=True)
    return X_data, y_data

In [4]:
from sklearn.model_selection import KFold
import pickle
import pandas as pd
# change: response (X) -> spectrum, spectra (y) -> temperature

spectrum_raw, temperature_raw = load_data(filename_dictionary=path_dictionary)
print(f"total number of examples:     {spectrum_raw.shape[1]}")
print(f"length of each example:       {spectrum_raw.shape[0]}")
print(f"shape of X data (spectrum): {spectrum_raw.shape}, type: {spectrum_raw[0][0].dtype}")
print(f"shape of y data (temperature): {temperature_raw.shape}, type: {temperature_raw[0].dtype}")
                                            

reading file:        5.9.23/twentypointseven
	temperature value: 20.7
reading file:        5.9.23/twentyonepointzero_1
	temperature value: 21.0
reading file:        5.9.23/twentyonepointtwo_1
	temperature value: 21.2
reading file:        5.9.23/twentyonepointfive_1
	temperature value: 21.5
reading file:        5.9.23/TwentyonepointseevendegreeC_1
	temperature value: 21.7
reading file:        5.9.23/twentypointeight_1
	temperature value: 21.8
total number of examples:     60000
length of each example:       1000
shape of X data (spectrum): (1000, 60000), type: float64
shape of y data (temperature): (1, 60000), type: float64


In [5]:
"""normalization: longer sklearn implementation"""
# more code, but can change type of scaler easily
from sklearn.preprocessing import StandardScaler
import random
scaler = StandardScaler() # change scaler here
index = random.randint(0, spectrum_raw.shape[0] - 1)
print(f"test with {index}th row")
print("normalized mean will not be exactly 0, but very close to it!")
spectrum = spectrum_raw.copy()
for i in range(spectrum_raw.shape[0]):
    # StandardScaler only takes 2d array
    array = np.reshape(np.asarray(spectrum_raw.iloc[i]), (-1, 1))
    transformed = scaler.fit_transform(array)
    spectrum.iloc[i] = np.reshape(transformed, -1)
print("SPECTRUM")
print(f"0th row: before normalization: {spectrum_raw.iloc[index]}")
print(f"mean = {sum(spectrum_raw.iloc[index])/spectrum_raw.shape[1]}")
print(f"0th row: after normalization: {spectrum.iloc[index]}")
print(f"mean = {sum(spectrum.iloc[index])/spectrum_raw.shape[1]}")
temperature = temperature_raw.copy()
for i in range(temperature_raw.shape[0]):
    # StandardScaler only takes 2d array
    array = np.reshape(np.asarray(temperature_raw.iloc[i]), (-1, 1))
    transformed = scaler.fit_transform(array)
    temperature.iloc[i] = np.reshape(transformed, -1)
print("TEMPERATURE")
print(f"0th row: before normalization: {temperature_raw.iloc[0]}")
print(f"mean = {sum(temperature_raw.iloc[0])/spectrum_raw.shape[1]}")
print(f"0th row: after normalization: {temperature.iloc[0]}")
print(f"mean = {sum(temperature.iloc[0])/spectrum_raw.shape[1]}")

test with 66th row
normalized mean will not be exactly 0, but very close to it!
SPECTRUM
0th row: before normalization: 0        0.816
1        0.832
2        0.856
3        0.872
4        0.864
         ...  
59995    0.888
59996    0.896
59997    0.888
59998    0.912
59999    0.912
Name: 66, Length: 60000, dtype: float64
mean = 0.9231629333332636
0th row: after normalization: 0       -1.105015
1       -0.940031
2       -0.692553
3       -0.527569
4       -0.610061
           ...   
59995   -0.362584
59996   -0.280092
59997   -0.362584
59998   -0.115107
59999   -0.115107
Name: 66, Length: 60000, dtype: float64
mean = -5.432357387998484e-14
TEMPERATURE
0th row: before normalization: 0        20.7
1        20.7
2        20.7
3        20.7
4        20.7
         ... 
59995    21.8
59996    21.8
59997    21.8
59998    21.8
59999    21.8
Name: 0, Length: 60000, dtype: float64
mean = 21.31666666666431
0th row: after normalization: 0       -1.584906
1       -1.584906
2       -1.584906
3     

In [6]:
# shorter pandas implementation, also work (very small difference in value)
# less code, but more changes needed to change normalization method
# spectrum = spectrum_raw.sub(spectrum_raw.mean(axis=1), axis=0).div(spectrum_raw.std(axis=1), axis=0)
# print(f"0th row: before normalization: {spectrum_raw.iloc[0]}")
# print(f"mean = {sum(spectrum_raw.iloc[0])/spectrum_raw.shape[1]}")
# print(f"0th row: after normalization: {spectrum.iloc[0]}")
# print(f"mean = {sum(spectrum.iloc[0])/spectrum_raw.shape[1]}")
# temperature = temperature_raw.sub(temperature_raw.mean(axis=1), axis=0).div(temperature_raw.std(axis=1), axis=0)
# print(f"0th row: before normalization: {temperature_raw.iloc[0]}")
# print(f"mean = {sum(temperature_raw.iloc[0])/spectrum_raw.shape[1]}")
# print(f"0th row: after normalization: {temperature.iloc[0]}")
# print(f"mean = {sum(temperature.iloc[0])/spectrum_raw.shape[1]}")

In [7]:
import os.path
file_name = 'cross_validation_resample=2_fold=5_dnn'
if os.path.isfile(file_name+'.pickle'): 
    with open(file_name+'.pickle', 'rb') as handle:
        train_indices,test_indices = pickle.load(handle)
    print(f"got indices from {file_name}.pickle")  
else:
    # 2x5 resampling
    train_indices = []
    test_indices = []
    number_resamples = 2
    n_splits =5
    for i in range(number_resamples):
        kf = KFold(n_splits=n_splits, random_state=i, shuffle=True)
        
        for i, (train_index, test_index) in enumerate(kf.split(range(temperature.shape[1]))):
            train_indices.append(train_index)
            test_indices.append(test_index)
    print(f"got indices by KFold method, fold = {n_splits}, resample = {number_resamples}")
    with open(file_name+'.pickle', 'wb') as handle:
        pickle.dump([train_indices,test_indices], handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"saved indices in {file_name}.pickle file")


got indices from cross_validation_resample=2_fold=5_dnn.pickle


In [8]:
print(f"sets of training indices: {len(train_indices)}")
print(f"number of training indices per set: {len(train_indices[0])}")
print(f"sets of testing indices: {len(test_indices)}")
print(f"number of testing indices per set: {len(test_indices[0])}")

sets of training indices: 10
number of training indices per set: 48000
sets of testing indices: 10
number of testing indices per set: 12000


In [9]:
import torch
import torch.nn as nn
# change: input dim = 1000, output dim = 1 (temperature value)
class Model(torch.nn.Module):
    def __init__(self,device, input_dim=1000):
        super().__init__()
        self.relu  = nn.ReLU()
        self.hidden_dim = 500
        self.linear1 = torch.nn.Linear(input_dim, self.hidden_dim)
        self.linear2= torch.nn.Linear(self.hidden_dim, self.hidden_dim)
        self.linear3= torch.nn.Linear(self.hidden_dim, 1)
        self.device = device
        self.to(device)
    def forward(self, x):
        y = self.linear3(self.relu(self.linear2(self.relu(self.linear1(x)))))
        # change: remove sigmoid, use y result as is
        # y = torch.sigmoid(y)
        return y


In [17]:
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CalculateMSE():
    def __init__(self, net, n_epochs, batch_size):
        super().__init__()
        self.net = net
        #initialize some constants
        self.batch_size = 32
        self.learning_rate = 1e-4
        self.n_epochs = n_epochs
        self.net.apply(self.weights_init)   
    def weights_init(self,layer):
        if type(layer) == nn.Linear:
            nn.init.orthogonal_(layer.weight)
    def get_mse(self,train_data, train_label, test_data, test_label):
        train_set = torch.utils.data.TensorDataset(
            torch.Tensor(train_data).to(device), 
            torch.Tensor(train_label).to(device))
        val_set = torch.utils.data.TensorDataset(
            torch.Tensor(test_data).to(device), 
            torch.Tensor(test_label).to(device))
        loader_args = dict(batch_size=self.batch_size)
        train_loader = DataLoader(train_set, shuffle=True, drop_last=True, **loader_args)
        val_loader = DataLoader(val_set, shuffle=True, drop_last=True, **loader_args)
        tloss = []
        vloss = []
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate) # weight_decay=0
        for epoch in range(0, self.n_epochs):
            # if epoch % 1000 == 0:
            #     print(f"epoch = {epoch}")
            epoch_train_loss=[]
            for i, data in enumerate(train_loader, 0):
                inputs, label = data
                y_pred = self.net(inputs.to(self.net.device))
                loss = criterion(y_pred, label.to(self.net.device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_train_loss.append(loss.item())
            tloss.append(np.mean(epoch_train_loss))
            epoch_loss=[]
            for i, data in enumerate(val_loader, 0):
                with torch.no_grad():
                    inputs1, label1 = data
                    y_pred1 = self.net(inputs1.to(self.net.device))
                    loss1 = criterion(y_pred1, label1.to(self.net.device))
                    epoch_loss.append(loss1.item())
            vloss.append(np.mean(epoch_loss))
        return np.min(vloss), self.net


In [22]:
from pathlib import Path
from datetime import datetime
# change: turn into 10 right now for development, was 3000
n_epochs=100
batch_size=32

PATH = 'model_dnn/'
Path(PATH).mkdir(parents=True, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# change: commented out: alraedy read in X, y data earlier
# response = pd.read_csv("1127_final_data/response.csv", header=None).values #input X
# spectra = pd.read_csv("1127_final_data/spectra.csv", header=None).values #ground truth label Y
# change: input dim = 1000
mdl = Model(device=device, input_dim=1000)
losses = []
print(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {mdl.device}")

f = open("log_dnn.txt", "w")
f.write("train START: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.write(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {device}\n")
for i,(train,test) in enumerate(zip(train_indices,test_indices)):
    print(f"we are on fold no.{i}")
    train_data, train_label= spectrum[train],temperature[train]
    test_data, test_label= spectrum[test],temperature[test]
    mse_calculator = CalculateMSE(mdl,n_epochs,batch_size)
    loss,model = mse_calculator.get_mse((np.asarray(train_data).T), 
                                        (np.asarray(train_label).T), 
                                        (np.asarray(test_data).T), 
                                        (np.asarray(test_label).T))
    losses.append(loss)
    print(f"\tloss: {loss}")
    print("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
    f.write(f"fold = {i}")
    f.write(f"\tloss: {loss}\n")
    f.write("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") +"\n")

    torch.save(model.state_dict(), PATH+'model_'+str(i))
f.write("train END: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.close()

number of epochs: 1, batch size: 32, device: cuda
we are on fold no.0
	loss: 0.33833638179302217
	time: 2023-05-18 09:01:30
we are on fold no.1
	loss: 0.2672790770928065
	time: 2023-05-18 09:01:39
we are on fold no.2
	loss: 0.3473200652996699
	time: 2023-05-18 09:01:48
we are on fold no.3
	loss: 0.24806785235802334
	time: 2023-05-18 09:01:57
we are on fold no.4
	loss: 0.22492081566651662
	time: 2023-05-18 09:02:07
we are on fold no.5
	loss: 0.22177571150660516
	time: 2023-05-18 09:02:16
we are on fold no.6
	loss: 0.1869705994526545
	time: 2023-05-18 09:02:25
we are on fold no.7
	loss: 0.2209548215866089
	time: 2023-05-18 09:02:34
we are on fold no.8
	loss: 0.16448056971033415
	time: 2023-05-18 09:02:43
we are on fold no.9
	loss: 0.18232015985250474
	time: 2023-05-18 09:02:52


In [12]:
# 230516: takes 78 minutes per fold! loss is 412
# After 700 minute, loss is:
# we are on fold no.0
# 	loss: 412.9017034505208
# we are on fold no.1
# 	loss: 412.9801847330729
# we are on fold no.2
# 	loss: 412.79992659505206
# we are on fold no.3
# 	loss: 412.94208658854166
# we are on fold no.4
# 	loss: 412.96778076171876
# we are on fold no.5
# 	loss: 412.6050344238281
# we are on fold no.6
# 	loss: 412.8362565917969
# we are on fold no.7
# 	loss: 413.03926790364585
# we are on fold no.8
# 	loss: 413.1065719401042
# we are on fold no.9
# 	loss: 413.0045514322917
# intuition, label is about 20, error is MSE, 
# so my predictions are always 0 or 1-ish!
# Problem: used sigmoid(y) at end of neural network!!!

In [13]:
print(f"mean losses: {np.mean(losses)}, std: {np.std(losses)}")

mean losses: 0.22448036237061028, std: 0.04588106152693985


In [14]:
number_figures = 10
import matplotlib.pyplot as plt

indices = torch.randint(0,len(spectrum),(number_figures,)).unique()
for i in indices:
    print(f"we use {i}th example")
    # change: cast i to int, since pandas not work with torch.int64
    # changed: removed figure, since the output is just one number
    spec = np.asarray(spectrum[int(i)]).flatten()
    temp = np.asarray(temperature[int(i)]).flatten()
    # plt.figure(i)

    prediction = model(torch.Tensor(np.asarray(spectrum[int(i)])).to(model.device)).detach().cpu().flatten()
    # plt.plot(prediction)
    print(f"\tthe prediction (normalized) is: {prediction.item()}")
    # plt.plot(temp)
    # plt.legend(['reconstruction','ground truth'])
    print(f"\tthe ground truth (normalized) is: {temp.item()}")


we use 162th example
	the prediction (normalized) is: -0.9298792481422424
	the ground truth (normalized) is: -1.5849058664490416
we use 180th example
	the prediction (normalized) is: -2.0472285747528076
	the ground truth (normalized) is: -1.5849058664490416
we use 356th example
	the prediction (normalized) is: -1.262682318687439
	the ground truth (normalized) is: -1.5849058664490416
we use 375th example
	the prediction (normalized) is: -1.2654244899749756
	the ground truth (normalized) is: -1.5849058664490416
we use 427th example
	the prediction (normalized) is: -1.097796082496643
	the ground truth (normalized) is: -1.5849058664490416
we use 501th example
	the prediction (normalized) is: -1.434181571006775
	the ground truth (normalized) is: -1.5849058664490416
we use 558th example
	the prediction (normalized) is: -1.610810399055481
	the ground truth (normalized) is: -1.5849058664490416
we use 566th example
	the prediction (normalized) is: -0.6301397681236267
	the ground truth (normaliz

In [15]:
from pathlib import Path
PATH = 'model_dnn/'
device = torch.device("cuda")
mdl = Model(device=device, input_dim=1000)

from scipy import stats,spatial
#pip install dtw-python
from dtw import * # <- this is dtw-python, NOT dtw package! (different dtw()s)
import torch
import numpy as np
# response = pd.read_csv("1127_final_data/response.csv", header=None).values #input X
# spectra = pd.read_csv("1127_final_data/spectra.csv", header=None).values #ground truth label Y
correlation_losses = []

print("I calculate coefficients like pearsonr with respect to all test examples and their labels")
def calculate_correlation(model, test_data, test_label):
    test_data_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)
    # print(f"shape of test_data_tensor: {test_data_tensor.shape}")
    construction = model(test_data_tensor).detach().cpu().numpy()
    # print(f"shape of construction: {construction.shape}")
    # Pearson
    pearson_coefs = []
    pearson_ps = []
    
    # Kendall
    kendall_coefs = []
    kendall_ps = []
    
    # Spearman
    spearman_coefs = []
    spearman_ps = []
    
    # Distance Correlation
    distance_corr = []
    
    #DTW distance
    alignment = []
    
    #absolute_error
    abs_err = []
    
    # print(f"test_label.shape[0]: {test_label.shape[0]}")
    for i in range(test_label.shape[1]):
        x1 = np.reshape(construction, -1)
        # print(f"shape of x1: {x1.shape}")
        x2 = np.reshape(test_label, -1)
        # print(f"shape of x2: {x2.shape}")
        res = stats.pearsonr(x1, x2)
        pearson_coefs.append(res[0])
        pearson_ps.append(res[1])
        
        res = stats.kendalltau(x1, x2)
        kendall_coefs.append(res[0])
        kendall_ps.append(res[1])
        
        res = stats.spearmanr(x1, x2)
        spearman_coefs.append(res[0])
        spearman_ps.append(res[1])
        
        distance_corr.append(1- spatial.distance.correlation(x1,x2))

        # change: removed dtw <- this is for time series, 
        # time is not involved in this project (I think!)
        # alignment.append(dtw(x1, x2, distance_only=True).distance)
        abs_err.append(abs(x1-x2))
        
    correlation_results = {
        'pearson': (pearson_coefs, pearson_ps),
        'kendall': (kendall_coefs, kendall_ps),
        'spearman': (spearman_coefs, spearman_ps),
        # 'DTW': alignment,
        'Absolute Error': abs_err,
        'Distance Correlation': distance_corr
    }

    return correlation_results

for i, (train, test) in enumerate(zip(train_indices, test_indices)):
    print(i)
    train_data, train_label = spectrum[train], temperature[train]
    test_data, test_label = spectrum[test], temperature[test]
    
    mdl_name = PATH + 'model_' + str(i)
    mdl.load_state_dict(torch.load(mdl_name))
    mdl.eval()
    
    correlation_loss = calculate_correlation(mdl, 
                                             np.transpose(np.asarray(test_data)), 
                                             np.transpose(np.asarray(test_label)))
    correlation_losses.append(correlation_loss)
for key in correlation_losses[0].keys():
    print(key)
    if key=='Absolute Error':
        errors = []
        for d in correlation_losses:
            errors+=np.concatenate(d[key]).ravel().tolist()
        #percentile
        percentiles = [5, 50, 90, 95, 99]
        for p in percentiles:
            print(p)
            print(np.percentile(errors, p))
    else:
        stat, p = [], []
        for d in correlation_losses:
            if key=='DTW' or key=='Distance Correlation':
                stat+=d[key]
            else:
                stat+=d[key][0]
                p+=d[key][1]
        print(np.mean(stat),np.std(stat))
        if len(p)>0:
            print(np.mean(p),np.std(p))

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

I calculate coefficients like pearsonr with respect to all test examples and their labels
0
1
2
3
4
5
6
7
8
9
pearson
0.8847269879241996 0.023893769074949393
0.0 0.0
kendall
0.7493965290947531 0.030957607682910573
0.0 0.0
spearman
0.8906862179539354 0.020438000456394396
0.0 0.0
Absolute Error
5
0.020276281057922042
50
0.2325447021345507
90
0.8002219724030599
95
1.0396681156649568
99
1.5282319440592955
Distance Correlation
0.884726990705244 0.023893778790746403
